In [1]:
import tweepy
import pandas as pd
import numpy as np
import csv
import re
pd.set_option("display.max_colwidth", None)
from collections import Counter
import scipy.sparse as sp
from nltk.corpus import stopwords
stopwords = stopwords.words('english')
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import itertools

pre-processing

In [2]:
path=r'D:\华威Warwick\class\project\tweets'   #load data
dem1 = pd.read_csv(path+'\dems_tweets.csv')
dem1 = dem1[['user_id','Time','Text']]
dem2 = pd.read_csv(path+'\dems_tweets_2.csv')
dem2 = dem2[['user_id','Time','Text']]
dems = pd.concat([dem1,dem2])
dems = dems.reset_index()
dems = dems[['user_id','Time','Text']]

rep1 = pd.read_csv(path+'\\reps_tweets.csv')
rep1 = rep1[['user_id','Time','Text']]
rep2 = pd.read_csv(path+'\\reps_tweets_2.csv')
rep2 = rep2[['user_id','Time','Text']]
reps = pd.concat([rep1,rep2])
reps = reps.reset_index()
reps = reps[['user_id','Time','Text']]

In [3]:
print(len(reps))
print(len(dems))

1314749
1456621


In [3]:
reps = reps[reps["Time"] > '2020/9/26']  #filter date
reps = reps[reps["Time"] < '2021/7/26']

dems = dems[dems["Time"] > '2020/9/26']
dems = dems[dems["Time"] < '2021/7/26']


In [4]:
# first preprecessing
dems['Text'] = dems['Text'].apply(lambda x:x[2:])  #edit raw text
reps['Text'] = reps['Text'].apply(lambda x:x[2:])
dems['Text'] = dems['Text'].apply(lambda x:x[:-1])
reps['Text'] = reps['Text'].apply(lambda x:x[:-1])

dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x99', '\'') #replace some unicodes
dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9c', '\"')
dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9d', '\"')
reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x99', '\'')
reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x9c', '\"')
reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x9d', '\"')

dems['Text'] = dems['Text'].str.replace(r'\\x[A-Za-z0-9./]+', '')
reps['Text'] = reps['Text'].str.replace(r'\\x[A-Za-z0-9./]+', '')#remove all other unicodes

dems['Text'] = dems['Text'].str.replace(r'\\n', ' ')
reps['Text'] = reps['Text'].str.replace(r'\\n', ' ')

<ipython-input-4-09800e05c008>:7: FutureWarning: The default value of regex will change from True to False in a future version.
  dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x99', '\'') #replace some unicodes
<ipython-input-4-09800e05c008>:8: FutureWarning: The default value of regex will change from True to False in a future version.
  dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9c', '\"')
<ipython-input-4-09800e05c008>:9: FutureWarning: The default value of regex will change from True to False in a future version.
  dems['Text'] = dems['Text'].str.replace(r'\\xe2\\x80\\x9d', '\"')
<ipython-input-4-09800e05c008>:10: FutureWarning: The default value of regex will change from True to False in a future version.
  reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x99', '\'')
<ipython-input-4-09800e05c008>:11: FutureWarning: The default value of regex will change from True to False in a future version.
  reps['Text'] = reps['Text'].str.replace(r'\\xe2\\x80\\x9c', '

In [5]:
#regular expressions
dems['Text'] = dems.Text.str.lower()
dems['Text'] = dems.Text.apply(lambda x: re.sub(r'\b(lol)\b', 'happyface', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', x))
dems['Text']= dems.Text.apply(lambda x: re.sub("@[A-Za-z0-9_]+", ' ', x))
dems['Text']= dems.Text.apply(lambda x: re.sub("#[A-Za-z0-9_]+", ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"\b[0-9]+\b", ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"[^A-Za-z0-9 ]",'', x))
dems['Text']= dems.Text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(r"\b[a-z0-9]\b",' ', x))
dems['Text'] = dems.Text.apply(lambda x: re.sub(' +', ' ', x))

reps['Text'] = reps.Text.str.lower()
reps['Text'] = reps.Text.apply(lambda x: re.sub(r'\b(lol)\b', 'happyface', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r'https?:\/\/\S+', ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"www\.[a-z]?\.?(com)+|[a-z]+\.(com)", ' ', x))
reps['Text']= reps.Text.apply(lambda x: re.sub("@[A-Za-z0-9_]+", ' ', x))
reps['Text']= reps.Text.apply(lambda x: re.sub("#[A-Za-z0-9_]+", ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"\b[0-9]+\b", ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"[^A-Za-z0-9 ]",'', x))
reps['Text']= reps.Text.apply(lambda x: re.sub(r"[^a-z\s\(\-:\)\\\/\];='#]", ' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(r"\b[a-z0-9]\b",' ', x))
reps['Text'] = reps.Text.apply(lambda x: re.sub(' +', ' ', x))


In [6]:
reps = reps[reps["Text"] != ' '];reps = reps[reps["Text"] != '']
dems = dems[dems["Text"] != ' '];dems = dems[dems["Text"] != '']
reps.index = range(len(reps));dems.index = range(len(dems))

In [84]:
def preprocess_tweet_text(tweet):
    tweet_tokens = word_tokenize(tweet)
    filtered_words = [w for w in tweet_tokens if not w in stopwords]
    ps = PorterStemmer()
    stemmed_words = [ps.stem(w) for w in filtered_words]
    lemmatizer = WordNetLemmatizer()
    lemma_words = [lemmatizer.lemmatize(w, pos='a') for w in stemmed_words]
    return lemma_words
reps['Text'] = reps.Text.apply(lambda x:preprocess_tweet_text(x))
dems['Text'] = dems.Text.apply(lambda x:preprocess_tweet_text(x))

In [92]:
# List of all words across tweets
all_tokens = list(itertools.chain(*dems['Text']))
# Create counter
token_counts = Counter(all_tokens)
token_counts.most_common(15)


[('trump', 178074),
 ('peopl', 111280),
 ('vote', 111247),
 ('amp', 107581),
 ('get', 95714),
 ('us', 90579),
 ('one', 77918),
 ('like', 76685),
 ('republican', 74546),
 ('right', 72205),
 ('need', 70817),
 ('presid', 69854),
 ('say', 66718),
 ('go', 65382),
 ('american', 64747)]

topic mining done in Google Colab

In [149]:
#save topic dictionary from Google Colab 
np.save(r'D:\华威Warwick\class\project\topic\reps_topics.npy', reps_topics) 
#Load
reps_topics = np.load(r'D:\华威Warwick\class\project\topic\reps_topics.npy',allow_pickle=True).item()


In [146]:
import pickle #save topic frequency
with open(r'D:\华威Warwick\class\project\topic\reps_topics_freq', 'wb') as fp:
    pickle.dump(reps_topics_freq, fp)
with open (r'D:\华威Warwick\class\project\topic\reps_topics_freq', 'rb') as fp:
    reps_topics_freq = pickle.load(fp)

In [156]:
import pickle #save topic frequency
#with open(r'D:\华威Warwick\class\project\topic\dems_topics_freq', 'wb') as fp:
#    pickle.dump(dems_topics_freq, fp)
with open (r'D:\华威Warwick\class\project\topic\dems_topics_freq', 'rb') as fp:
    dems_topics_freq = pickle.load(fp)

In [154]:
#save topic dictionary from Google Colab 
np.save(r'D:\华威Warwick\class\project\topic\dems_topics.npy', dems_topics) 
#Load
dems_topics = np.load(r'D:\华威Warwick\class\project\topic\dems_topics.npy',allow_pickle=True).item()

 examples after topic modeling

In [153]:
dems_topics[97]

[('china', 0.1660926476014176),
 ('chinese', 0.08281270387423889),
 ('hong', 0.045425947238850115),
 ('hong kong', 0.04211122871988331),
 ('kong', 0.04140854079393119),
 ('chinas', 0.030206990181393042),
 ('tariffs', 0.028949925565671105),
 ('korea', 0.024627890473090917),
 ('north korea', 0.02213193145417531),
 ('xi', 0.014078272764133185),
 ('taiwan', 0.011349444418536504),
 ('human rights', 0.011016273214463506),
 ('government', 0.01076760955393996),
 ('war', 0.00827780366914058),
 ('qatar', 0.0082094720975641),
 ('allies', 0.008192164607000353),
 ('doesnt pay', 0.008061763786086909),
 ('russia', 0.00783571512715824),
 ('genocide', 0.007713403216593489),
 ('camps', 0.007572924209583971)]

In [83]:
reps_topics[7]

[('china', 0.07941592246338233),
 ('chinese', 0.04705733900178468),
 ('communist', 0.023257997849948794),
 ('chinas', 0.021494693488372855),
 ('chinese communist', 0.019696482923150672),
 ('communist party', 0.018439374273802007),
 ('hong', 0.014872816836239835),
 ('taiwan', 0.01415692305496534),
 ('beijing', 0.01383700143373067),
 ('hong kong', 0.012785234755498517),
 ('kong', 0.01263927907945536),
 ('communist china', 0.010656882321142636),
 ('xi', 0.009754664582997738),
 ('virus', 0.009257583480505328),
 ('human rights', 0.009021979530270672),
 ('chinese spy', 0.008995125774581877),
 ('china virus', 0.006516586173291237),
 ('rights', 0.00634682742651267),
 ('chinese government', 0.006133870036125793),
 ('beijing biden', 0.0055071164538115615)]